In [26]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


In [27]:
df = pd.read_csv("../Data/cleaned_data.csv")

In [28]:
X = df.drop(columns=['drug_name'])  
y = df['drug_name']
print(y.shape)

(10777,)


In [ ]:
cols = ["drug_target", "target_pathway", "feature_name", "tissue_type", "screening_set"]
le = LabelEncoder()

for col in cols:
    X[col] = le.fit_transform(X[col].astype(str))
    
y_encoded = le.fit_transform(y)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train = X_train[:8621]
y_train = y_train[:8621]
print(X_train.shape)  
print(y_train.shape)

(8621, 21)
(8621,)


In [34]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_encoded)

XGBoostError: [09:50:34] C:\actions-runner\_work\xgboost\xgboost\src\data\data.cc:542: Check failed: this->labels.Size() % this->num_row_ == 0 (2156 vs. 0) : Incorrect size for labels: (10777,1) v.s. 8621